importamos librerias

In [110]:
import numpy as np

import os

import re

import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report


impotamos liberias e KERAS


In [111]:
!pip3 install -q --user --upgrade tfp-nightly
import tensorflow_probability as tfp

In [112]:
pip install keras-layer-normalization


In [113]:

import keras

from keras.utils import to_categorical

from keras.models import Sequential,Model

from tensorflow.keras.layers import Input

from keras.layers import Dense, Dropout, Flatten

from keras.layers import Conv2D, MaxPooling2D

from keras.layers import BatchNormalization

from keras.layers import LeakyReLU

In [ ]:
dirname = os.path.join(os.getcwd(), 'ninhosimages')

imgpath = dirname + os.sep



images = []

directories = []

dircount = []

prevRoot=''

cant=0



for root, dirnames, filenames in os.walk(imgpath):

    for filename in filenames:

        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):

            cant=cant+1

            filepath = os.path.join(root, filename)

            image = plt.imread(filepath)

            images.append(image)

            b = "Leyendo..." + str(cant)

            if prevRoot !=root:

                prevRoot=root

                directories.append(root)

                dircount.append(cant)

                cant=0

dircount.append(cant)



dircount = dircount[1:]

dircount[0]=dircount[0]+1

print("Directorios leidos: ",len(directories))

print("Imagenes en cada directorio: ", dircount)

print("Total de imagenes en subdirectorio: ",sum(dircount))

In [128]:
print("Número de muestras en X:", len(X))
print("Número de muestras en y:", len(y))

Número de muestras en X: 1
Número de muestras en y: 1


In [134]:
labels=[]

indice=0

for cantidad in dircount:

    for i in range(cantidad):

        labels.append(indice)

    indice=indice+1

print("Cantidad etiquetas creadas: ",len(labels))



ninhos=[]

indice=0

for directorio in directories:

    name = directorio.split(os.sep)

    print(indice , name[len(name)-1])

    ninhos.append(name[len(name)-1])

    indice=indice+1



y = np.array(labels)

X = np.array(images, dtype=np.uint8) #convierto de lista a numpy uint8



# Buscar los números únicos de las etiquetas del aprendizaje

classes = np.unique(y)

nClasses = len(classes)

print('Total de outputs : ', nClasses)

print('Output classes : ', classes)

Cantidad etiquetas creadas:  1
0 
Total de outputs :  1
Output classes :  [0]


In [133]:
#Mezclar todo y crear los grupos de entrenamiento y testing


train_X, test_X, train_Y, test_Y = train_test_split(X, y, test_size=0.2)



print('Datos de aprendizaje : ', train_X.shape, train_Y.shape)

print('Testing de aprendizaje : ', test_X.shape, test_Y.shape)



plt.figure(figsize=[5,5])



# Display the first image in training data

plt.subplot(121)

plt.imshow(train_X[0,:,:], cmap='gray')

plt.title("Ground Truth : {}".format(train_Y[0]))



# Display the first image in testing data

plt.subplot(122)

plt.imshow(test_X[0,:,:], cmap='gray')

plt.title("Ground Truth : {}".format(test_Y[0]))

ValueError: ignored

In [ ]:
train_X = train_X.astype('float32')

test_X = test_X.astype('float32')

train_X = train_X / 255.

test_X = test_X / 255.



# Change the labels from categorical to one-hot encoding

train_Y_one_hot = to_categorical(train_Y)

test_Y_one_hot = to_categorical(test_Y)



# Display the change for category label using one-hot encoding

print('Etiqueta original:', train_Y[0])

print('Después de la conversión a one-hot:', train_Y_one_hot[0])

In [ ]:
#Mezclar todo y crear los grupos de entrenamiento y testing

train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)



print(train_X.shape,valid_X.shape,train_label.shape,valid_label.shape)

In [ ]:
#declaramos variables con los parámetros de configuración de la red

INIT_LR = 1e-3

epochs = 6

batch_size = 64



ninhos_model = Sequential()

ninhos_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(21,28,3)))

ninhos_model.add(LeakyReLU(alpha=0.1))

ninhos_model.add(MaxPooling2D((2, 2),padding='same'))

ninhos_model.add(Dropout(0.5))



ninhos_model.add(Flatten())

ninhos_model.add(Dense(32, activation='linear'))

ninhos_model.add(LeakyReLU(alpha=0.1))

ninhos_model.add(Dropout(0.5))

ninhos_model.add(Dense(nClasses, activation='softmax'))



ninhos_model.summary()



ninhos_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adagrad(lr=INIT_LR, decay=INIT_LR / 100),metrics=['accuracy'])

In [ ]:
# Iniciar entrenamiento

# Este paso puede tomar varios minutos, dependiendo del cpu y memoria ram

ninhos_train = ninhos_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))



# guardamos la red, para reutilizarla en el futuro, sin tener que volver a entrenar

ninhos_model.save("ninhos_mnist.h5py")

In [ ]:
test_eval = ninhos_model.evaluate(test_X, test_Y_one_hot, verbose=1)



print('TEST:')

print('Perdida:', test_eval[0])

print('Exactitud:', test_eval[1])



accuracy = ninhos_train.history['accuracy']

val_accuracy = ninhos_train.history['val_accuracy']

loss = ninhos_train.history['loss']

val_loss = ninhos_train.history['val_loss']

epochs = range(len(accuracy))

plt.plot(epochs, accuracy, 'bo', label='Precision de entrenamiento')

plt.plot(epochs, val_accuracy, 'b', label='Exactitud de la validacion')

plt.title('Precisión de entrenamiento y validación')

plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Perdida de entrenamiento')

plt.plot(epochs, val_loss, 'b', label='Perdida de validacion')

plt.title('Perdida de entrenamiento y validacion')

plt.legend()

plt.show()







predicted_classes2 = ninhos_model.predict(test_X)



predicted_classes=[]

for predicted_ninhos in predicted_classes2:

    predicted_classes.append(predicted_ninhos.tolist().index(max(predicted_ninhos)))

predicted_classes=np.array(predicted_classes)



predicted_classes.shape, test_Y.shape